In [3]:
import pandas as pd
import numpy as np
import os
import glob

# ==========================================
# KONFIGURASI
# ==========================================
FOLDER_PATH = 'idx30/'   # Pastikan path folder benar
FILE_IHSG = 'IHSG.csv'        # Nama file IHSG
WINDOW_VOLATILITAS = 3        # Rolling window 3 bulan

# ==========================================
# FUNGSI CLEANING (SMART DATE & NUMBER PARSING)
# ==========================================
def clean_investing_data(file_path, is_market_index=False):
    filename = os.path.basename(file_path)
    emiten_code = os.path.splitext(filename)[0]
    
    # 1. BACA SEMUA SEBAGAI STRING AGAR AMAN
    df = pd.read_csv(file_path, dtype=str)
    
    # Handling nama kolom Volume yang tidak konsisten
    col_vol = 'Vol.' if 'Vol.' in df.columns else 'Volume'
    
    # ---------------------------------------------------------
    # 2. DETEKSI FORMAT (US vs INDO)
    # ---------------------------------------------------------
    # Ambil sampel baris pertama yang tidak kosong
    sample_vol = str(df[col_vol].iloc[0]) if not df[col_vol].isna().all() else ""
    sample_price = str(df['Terakhir'].iloc[0])
    
    is_indo_format = False
    
    # Logika Deteksi:
    if ',' in sample_vol: 
        # Volume "1,88B" -> Format Indo
        is_indo_format = True
    elif '.' in sample_vol and 'B' in sample_vol:
        # Volume "3.74B" -> Format US
        is_indo_format = False
    else:
        # Fallback cek harga jika volume ambigu (misal integer murni)
        if '.' in sample_price and ',' not in sample_price:
            # Harga "1.810" (Titik = Ribuan) -> Indo
            is_indo_format = True
        elif ',' in sample_price:
            # Harga "3,680" (Koma = Ribuan) -> US
            is_indo_format = False

    # ---------------------------------------------------------
    # 3. PARSING TANGGAL & ANGKA SESUAI FORMAT
    # ---------------------------------------------------------
    if is_indo_format:
        # --- FORMAT INDO ---
        # Tanggal: d/m/y (Day First) -> 01/11/2025 adalah 1 November
        df['Tanggal'] = pd.to_datetime(df['Tanggal'], dayfirst=True, errors='coerce')
        
        # Harga: "1.810,50" -> Hapus titik, ganti koma jadi titik -> "1810.50"
        df['Terakhir'] = df['Terakhir'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
        
        # Volume: "1,88B" -> Ganti koma jadi titik -> "1.88B"
        if not is_market_index:
            df[col_vol] = df[col_vol].str.replace(',', '.', regex=False)
            
    else:
        # --- FORMAT US ---
        # Tanggal: m/d/y (Month First) -> 11/01/2025 adalah 1 November
        df['Tanggal'] = pd.to_datetime(df['Tanggal'], dayfirst=False, errors='coerce')
        
        # Harga: "3,680.50" -> Hapus koma -> "3680.50"
        df['Terakhir'] = df['Terakhir'].str.replace(',', '', regex=False)
        
        # Volume: "3.74B" -> Sudah pakai titik, hapus koma jaga-jaga
        if not is_market_index:
            df[col_vol] = df[col_vol].str.replace(',', '', regex=False)

    # Convert Harga ke Float
    df['Terakhir'] = df['Terakhir'].astype(float)
    
    # Urutkan berdasarkan tanggal yang sudah benar
    df = df.sort_values('Tanggal', ascending=True)

    # ---------------------------------------------------------
    # 4. PERHITUNGAN VARIABEL
    # ---------------------------------------------------------
    # Hitung Return
    df['return'] = np.log(df['Terakhir'] / df['Terakhir'].shift(1))
    
    if not is_market_index:
        # Cleaning Volume Unit (B/M/K)
        def parse_volume(vol_str):
            if pd.isna(vol_str) or vol_str == '-' or vol_str == 'nan':
                return np.nan
            
            vol_str = str(vol_str).upper()
            multiplier = 1
            
            if 'B' in vol_str:
                multiplier = 1_000_000_000
                vol_str = vol_str.replace('B', '')
            elif 'M' in vol_str:
                multiplier = 1_000_000
                vol_str = vol_str.replace('M', '')
            elif 'K' in vol_str:
                multiplier = 1_000
                vol_str = vol_str.replace('K', '')
            
            try:
                return float(vol_str) * multiplier
            except ValueError:
                return np.nan
        
        df['volume_clean'] = df[col_vol].apply(parse_volume)
        
        # Hitung Volatilitas (Rolling)
        df['volatilitas'] = df['return'].rolling(window=WINDOW_VOLATILITAS).std()
        
        df['emiten'] = emiten_code
        final_df = df[['Tanggal', 'emiten', 'Terakhir', 'volume_clean', 'return', 'volatilitas']].copy()
        final_df.columns = ['date', 'emiten', 'close', 'volume', 'stock_return', 'volatility']
        
    else:
        # IHSG
        final_df = df[['Tanggal', 'return']].copy()
        final_df.columns = ['date', 'market_return']

    return final_df

# ==========================================
# PROSES MERGING
# ==========================================

# 1. Load IHSG
path_ihsg = os.path.join(FOLDER_PATH, FILE_IHSG)
if os.path.exists(path_ihsg):
    print(f"Memproses Data Pasar: {FILE_IHSG}...")
    df_market = clean_investing_data(path_ihsg, is_market_index=True)
else:
    raise FileNotFoundError(f"File IHSG tidak ditemukan: {path_ihsg}")

# 2. Load Semua Saham
all_files = glob.glob(os.path.join(FOLDER_PATH, "*.csv"))
list_dataframes = []

print("Memulai proses merging data saham...")

for file in all_files:
    if os.path.basename(file) == FILE_IHSG:
        continue
        
    try:
        df_emiten = clean_investing_data(file, is_market_index=False)
        
        # Merge dengan IHSG
        df_merged = pd.merge(df_emiten, df_market, on='date', how='left')
        
        list_dataframes.append(df_merged)
        print(f"✅ Berhasil: {os.path.basename(file)}")
        
    except Exception as e:
        print(f"❌ Gagal {os.path.basename(file)}: {e}")

# 3. Gabung & Simpan
if list_dataframes:
    df_panel = pd.concat(list_dataframes, axis=0)
    df_final = df_panel.dropna()
    
    # Reset index agar rapi
    df_final = df_final.sort_values(['emiten', 'date']).reset_index(drop=True)
    
    print("\n=== HASIL AKHIR ===")
    print(f"Total Baris: {len(df_final)}")
    # Cek tanggal unik untuk memastikan bulanan
    print(f"Sample Tanggal: {df_final['date'].dt.strftime('%Y-%m-%d').unique()[:5]}")
    print("-" * 30)
    print(df_final.head())
    
    df_final.to_csv('Data_Panel_Final_Siap_Regresi.csv', index=False)
    print("\nFile disimpan: Data_Panel_Final_Siap_Regresi.csv")
else:
    print("Tidak ada data yang berhasil diproses.")

Memproses Data Pasar: IHSG.csv...
Memulai proses merging data saham...
✅ Berhasil: ADRO.csv
✅ Berhasil: AMRT.csv
✅ Berhasil: ANTM.csv
✅ Berhasil: ARTO.csv
✅ Berhasil: ASII.csv
✅ Berhasil: BBCA.csv
✅ Berhasil: BBNI.csv
✅ Berhasil: BBRI.csv
✅ Berhasil: BMRI.csv
✅ Berhasil: BRPT.csv
✅ Berhasil: CPIN.csv
✅ Berhasil: ELSA.csv
✅ Berhasil: ERAA.csv
✅ Berhasil: HEXA.csv
✅ Berhasil: ICBP.csv
✅ Berhasil: INDF.csv
✅ Berhasil: ITMG.csv
✅ Berhasil: KLBF.csv
✅ Berhasil: MDKA.csv
✅ Berhasil: MEDC.csv
✅ Berhasil: MIKA.csv
✅ Berhasil: PGAS.csv
✅ Berhasil: PTBA.csv
✅ Berhasil: SCMA.csv
✅ Berhasil: SMGR.csv
✅ Berhasil: TLKM.csv
✅ Berhasil: TOWR.csv
✅ Berhasil: UNTR.csv
✅ Berhasil: UNVR.csv

=== HASIL AKHIR ===
Total Baris: 2001
Sample Tanggal: ['2020-03-01' '2020-04-01' '2020-05-01' '2020-06-01' '2020-07-01']
------------------------------
        date emiten   close        volume  stock_return  volatility  \
0 2020-03-01   ADRO   990.0  1.540000e+09     -0.154151    0.089902   
1 2020-04-01   ADRO   920